In [2]:
import os
import shutil
import numpy as np
import pandas as pd

from tqdm import tqdm
from shutil import copyfile

import json 
import numpy as np 
from sklearn.model_selection import StratifiedGroupKFold 

In [27]:
# 원본 이미지 경로 저장  
img_PATH = './images/train/'
txt_PATH = './labels/train/'
 
list_img=[img for img in os.listdir(img_PATH) if img.endswith('.jpg')==True]
list_txt=[img for img in os.listdir(txt_PATH) if img.endswith('.txt')==True]

path_img=[]

for i in range (len(list_img)):
    path_img.append(img_PATH+list_img[i])
    
df=pd.DataFrame(path_img)
df = df.sort_values(by=[0]).reset_index(drop=True)


In [59]:
#https://stages.ai/competitions/178/discussion/talk/post/1205 참고


# load json 
annotation = "./train.json"
NUM_FOLD = 5
SEED = 42

with open(annotation) as f: data = json.load(f)

var = [(ann['image_id'], ann['category_id']) for ann in data['annotations']]
X = np.ones((len(data['annotations']),1))
y = np.array([v[1] for v in var])
groups = np.array([v[0] for v in var]) 

cv = StratifiedGroupKFold(n_splits=NUM_FOLD, shuffle=True, random_state=SEED) 

for train_idx, val_idx in (cv.split(X, y, groups)):
    NUM_FOLD -= 1
    print("TRAIN:", groups[train_idx])
    # print(" ", y[train_idx])
    # print(" TEST:", groups[val_idx])
    # print(" ", y[val_idx])
    try:
        shutil.rmtree(f'dataset_folds_{NUM_FOLD}/images')
        shutil.rmtree(f'dataset_folds_{NUM_FOLD}/labels')
    except:
        print('No dirs')

    # Make new dirs
    os.makedirs(f'dataset_folds_{NUM_FOLD}/images/train', exist_ok=True)
    os.makedirs(f'dataset_folds_{NUM_FOLD}/images/valid', exist_ok=True)
    os.makedirs(f'dataset_folds_{NUM_FOLD}/labels/train', exist_ok=True)
    os.makedirs(f'dataset_folds_{NUM_FOLD}/labels/valid', exist_ok=True)

    
    # Move the images to relevant split folder.
    for i in tqdm(groups[train_idx]):
        idx = str(10000 + i)[1:]
        copyfile(df.loc[i][0], f'dataset_folds_{NUM_FOLD}/images/train/{idx}.jpg')
        copyfile("./labels" + df.loc[i][0][8:-4] + ".txt", f'dataset_folds_{NUM_FOLD}/labels/train/{idx}.txt')
        
    for i in tqdm(groups[val_idx]):
        idx = str(10000 + i)[1:]
        copyfile(df.loc[i][0], f'dataset_folds_{NUM_FOLD}/images/valid/{idx}.jpg')
        copyfile("./labels" + df.loc[i][0][8:-4] + ".txt", f'dataset_folds_{NUM_FOLD}/labels/valid/{idx}.txt')

TRAIN: [   0    1    1 ... 4882 4882 4882]
No dirs


100%|██████████| 4336/4336 [00:06<00:00, 661.68it/s]


TRAIN: [   0    1    1 ... 4882 4882 4882]
No dirs


100%|██████████| 5092/5092 [00:07<00:00, 723.71it/s]


TRAIN: [   1    1    1 ... 4882 4882 4882]
No dirs


100%|██████████| 4706/4706 [00:06<00:00, 721.52it/s]


TRAIN: [   0    2    4 ... 4881 4881 4881]
No dirs


100%|██████████| 4571/4571 [00:06<00:00, 697.77it/s]


TRAIN: [   0    1    1 ... 4882 4882 4882]
No dirs


100%|██████████| 4439/4439 [00:06<00:00, 700.63it/s]


In [60]:
# Create .yaml file 
import yaml
NUM_FOLD = 5

for fold in range(NUM_FOLD):
    data_yaml = dict(
        train = f'../dataset/dataset_folds_{fold}/images/train',
        val = f'../dataset/dataset_folds_{fold}/images/valid',
        nc = 10,
        names = ["General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]  # class names
    )

    # Note that I am creating the file in the yolov5/data/ directory.
    with open(f'../yolov5/data/data_fold_{fold}.yaml', 'w') as outfile:
        yaml.dump(data_yaml, outfile, default_flow_style=True)
    
%cat ../yolov5/data/data_fold_0.yaml

{names: [General trash, Paper, Paper pack, Metal, Glass, Plastic, Styrofoam, Plastic
      bag, Battery, Clothing], nc: 10, train: ../dataset/dataset_folds_0/images/train,
  val: ../dataset/dataset_folds_0/images/valid}
